# NLP For Regression

In [79]:
import os
import numpy as np
import pandas as pd

from gensim import corpora
from gensim.models import TfidfModel, LdaModel
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, stem_text

from collections import defaultdict

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Let's bring in our guitar JSON parser:

In [29]:
from Axe_Object import Axe

And instantiate our rock n roll guitar objects:

In [30]:
filenames = [name for name in os.listdir('axe_specs/') if not name.startswith('.')] # Ignores hidden files on mac

In [31]:
axes = []
for filename in filenames:
    try:
        this_axe = Axe(filename)
        if "LOT OF" not in this_axe.title.upper() and this_axe.price_usd > 100 and this_axe.price_usd < 5000:
            if this_axe.string_config and this_axe.string_config < 5:
                continue
            axes.append(this_axe)
    except ValueError:
        pass

Check out all these text fields we can turn into a big nasty stew of NLP goodness:

In [32]:
axes[58].title

'1991 Japan Made Aria Pro II MAGNA Series Electric Guitar MA-550 Made in Japan'

In [33]:
axes[58].description 

"A.01 Fender PB62-75US vwh.html Ebay説明文原本 1991 Made Aria Pro II MAGNA Series Electric Guitar MA-550 Made in Japan Item\u3000Number ER-853 Condition Rank A No Reserve Auction! Shipping Free All over the world! USA Canada Buyer:Free shipping Europe Buyer: Free shipping Australia Buyer: Free shipping Brazil and South America buyer:Free shipping. We will ship this guitar No CITES application. We can ship this item within 1 business day After you made a payment. If the package will not able to any trouble. You can receive the full refund. no worry 1991 Made Aria Pro II MAGNA Series Electric Guitar MA-550 Made in Japan You're bidding on a very good working condition 1991 Made Aria Pro II MA-550, which contains very fine sound and rich tones made from good pieces of wood and hardworking craftsmanship. The guitar has slim neck shape with 24th frets type. Body has very slim comfortable shape. This is '90s made in Japan high quality made guitar. Tuner has GOTOH fine tuners. Sound is very nice. B

In [34]:
axes[58].condition_description # Might not exist for all instances

'Condition Rating(80/100)Not come with tremolo arm bar.Guitar has some scratches and dings on the body.(Photo-01)(Photo-02)The guitar is overall in very good working condition.The fret condition also very good(left 80%)(Not has a fret buzzing)Sounds and plays great with not issue.Please confirm pictures for details.'

In [35]:
axes[58].subtitle # Might not exist for all instances

'Japan Made vintage guitar No Reserve with FREE SHIPPING'

In [36]:
axes[58].brand # Might not exist for all instances

'OTHER'

In [37]:
axes[58].model # Might not exist for all instances

'MA-550'

In [38]:
str(axes[58].year) # Might not exist for all instances

'1991'

In [39]:
axes[58].material # Might not exist for all instances

'ALDER'

**Toss Together the Stew**

In [40]:
def assemble_guitar_document(axe):
    document = axe.title + ' '
    if axe.year != None:
        document += (str(axe.year) + ' ')*2
    if axe.material != None:
        document += axe.material + ' '
    if axe.model != None:
        document += axe.model + ' ' 
    if axe.brand != None:
        document += axe.brand + ' '
    if axe.subtitle != None:
        document += axe.subtitle + ' '
    if axe.condition_description != None:
        document += axe.condition_description + ' '
    if axe.description != None:
        document += axe.description
    return document

In [41]:
raw_corpus = [assemble_guitar_document(axe).lower() for axe in axes]

In [42]:
len(raw_corpus)

13134

**Text Pre-Processing:**

In [43]:
corpus = []
for doc in raw_corpus:
    doc = strip_multiple_whitespaces(strip_short(strip_punctuation(doc)))
    corpus.append([word for word in stem_text(remove_stopwords(doc)).split()])

In [44]:
# remove words that appear only once
frequency = defaultdict(int)
for doc in corpus:
    for word in doc:
        frequency[word] += 1
corpus = [[word for word in doc if frequency[word] > 1] for doc in corpus]

**Convert to Bag of Words:**

In [45]:
dictionary = corpora.Dictionary(corpus)

In [46]:
corpus = [dictionary.doc2bow(doc) for doc in corpus]

### LDA Transformation and Getting Weights:

In [49]:
def get_topic_weights_df(num_topics):
    
    lda = LdaModel(corpus, id2word=dictionary, num_topics=num_topics)
    
    corpus_lda = lda[corpus]

    blank_row = [0 for k in range(len(lda.print_topics(1001)))]
    to_concat = []

    for i in range(len(corpus_lda)):
        new_row = pd.Series(data=blank_row)
        for j in corpus_lda[i]:
            new_row.iloc[j[0]] = j[1]
        to_concat.append(new_row)
    
    return pd.concat(to_concat, axis=1).T

In [50]:
X = get_topic_weights_df(300)

### Linear Regression Baselining

Here I have to set up a primitive linear regression model to tune the # of topics we'll be using to feed the better one later.

In [51]:
prices = [axe.price_usd for axe in axes]
y = prices

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 42)

In [53]:
lin_reg = LinearRegression().fit(X_train, y_train)

In [54]:
y_preds = lin_reg.predict(X_test)

In [55]:
np.sqrt(mean_squared_error(y_test, y_preds))

541.4444132330826

In [56]:
price_mean = np.mean([axe.price_usd for axe in axes])
np.sqrt(mean_squared_error(y_test, [price_mean for i in range(len(y_test))]))

625.9088647093296

Incredibly, the text feature actually reduces error. But how many topics is best?

### Searching for ideal topic number

In [57]:
def get_ideal_num_topics(range_object):
    errors = []
    for i in range_object:
        X = get_topic_weights_df(i)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 42)
        lin_reg = LinearRegression().fit(X_train, y_train)
        y_preds = lin_reg.predict(X_test)
        error = np.sqrt(mean_squared_error(y_test, y_preds))
        errors.append(error)
    return errors

In [58]:
errors = get_ideal_num_topics(range(10,1010,10))

C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\models\ldamodel.py:1030: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\models\ldamodel.py:1030: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\models\ldamodel.py:1030: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\models\ldamodel.py:1030: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\models\ldamodel.py:1030: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\model

C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\models\ldamodel.py:1030: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\models\ldamodel.py:1030: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\models\ldamodel.py:1030: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\models\ldamodel.py:1030: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\models\ldamodel.py:1030: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\model

C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\models\ldamodel.py:1030: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [87]:
min(errors) # At 820 topics

520.6372335300139

In [88]:
get_ideal_num_topics(range(820,830,10))

C:\Users\rob\AppData\Local\conda\conda\envs\Bayes\lib\site-packages\gensim\models\ldamodel.py:1030: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


[536.7198446539411]